In [ ]:
from functools import partial
from pathlib import Path

import torch
import matplotlib.pyplot as plt
from pydantic import BaseModel, Field
from torch import Tensor
from torch.utils.data import DataLoader
from tqdm import tqdm

from ssd import SSD
from ssd.data import LetterboxTransform, SSDDataset
from ssd.structs import FrameLabels
from ssd.utils import TrainUtils
from ssd.utils.metrics_calculator import MetricsCalculator

In [ ]:
class EvaluateConfig(BaseModel):
    images_dir: Path
    labels_dir: Path
    min_confidence_threshold: float = Field(default=0.1)
    num_top_k: int = Field(default=100)
    nms_iou_threshold: float = Field(default=0.2)

### Define constants

In [ ]:
DEVICE = torch.device("cuda:0")
CONFIG = EvaluateConfig(
    images_dir=Path("/mnt/data/datasets/object_detection/coco/images/val2017"),
    labels_dir=Path("/mnt/data/datasets/object_detection/coco/labels/val2017"),
    min_confidence_threshold=0.1,
    num_top_k=100,
    nms_iou_threshold=0.2
)

MODEL_FILE = Path("/mnt/data/code/ssd/models/91f18512-9b06-4c9a-9d2c-8330ed7458c3/best.pt")
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
DTYPE = torch.float32

### Evaluate the model

In [ ]:
# Load the model in
model = SSD.load(MODEL_FILE, DEVICE)

In [ ]:
transform = LetterboxTransform(IMAGE_WIDTH, IMAGE_HEIGHT, DTYPE)
collate_func = partial(TrainUtils.batch_collate_func, device=DEVICE)

dataset = SSDDataset(CONFIG.images_dir, CONFIG.labels_dir, model.num_classes, transform, DEVICE, DTYPE)
data_loader = DataLoader(dataset, 8, shuffle=False, collate_fn=collate_func)

In [ ]:
model.eval()

images: Tensor
frame_labels: list[FrameLabels]
metrics_calculator = MetricsCalculator(model.num_classes)
image_detections: list[dict[str, Tensor]] = []
image_labels: list[dict[str, Tensor]] = []
for images, frame_labels in tqdm(data_loader):
    with torch.no_grad():
        head_outputs, anchors = model.forward(images)
        frame_detections = model._post_process_detections(head_outputs, anchors, CONFIG.min_confidence_threshold, CONFIG.num_top_k, CONFIG.nms_iou_threshold)

        metrics_calculator.update(frame_detections, frame_labels)

### Analyse per-class metrics

In [ ]:
CLASS_ID = 0

In [ ]:
# Plot the precision
precisions = metrics_calculator.precisions()[:, :, CLASS_ID]
precisions = precisions.cpu().numpy()

plt.figure()
for iou_idx in range(precisions.shape[1]):
    plt.plot(
        metrics_calculator._confidence_thresholds,
        precisions[:, iou_idx],
        label=f"IoU thresh = {metrics_calculator._iou_thresholds[iou_idx]:.2f}"
    )
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.title(f"Precision with confidence\nclass_id={CLASS_ID}")
plt.xlabel("Confidence threshold")
plt.ylabel("Precision")
plt.legend()
plt.grid()

In [ ]:
# Plot the recall
recalls = metrics_calculator.recalls()[:, :, CLASS_ID]
recalls = recalls.cpu().numpy()

plt.figure()
for iou_idx in range(precisions.shape[1]):
    plt.plot(
        metrics_calculator._confidence_thresholds,
        recalls[:, iou_idx],
        label=f"IoU thresh = {metrics_calculator._iou_thresholds[iou_idx]:.2f}"
    )
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.title(f"Recall with confidence\nclass_id={CLASS_ID}")
plt.xlabel("Confidence threshold")
plt.ylabel("Recall")
plt.legend()
plt.grid()

In [ ]:
# Create the PR-curve
plt.figure()
for iou_idx in range(precisions.shape[1]):
    plt.plot(
        recalls[:, iou_idx],
        precisions[:, iou_idx],
        label=f"IoU thresh = {metrics_calculator._iou_thresholds[iou_idx]:.2f}"
    )
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.title(f"Precision-recall curve\nclass_id={CLASS_ID}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.grid()

### Analyse overall accuracy

In [ ]:
mAPs = metrics_calculator.mAPs().cpu().numpy()

plt.figure()
class_ids = [cid for cid in range(model.num_classes)]
plt.bar(class_ids, mAPs)
plt.xlabel("Class ID")
plt.ylabel("mAP@(50-95)")
plt.grid()
plt.xlim((0, model.num_classes))
plt.ylim((0, 1))